In [8]:
import sys
sys.path.append("/home/schmid/Text2Head/NPHM")

import torch
import matplotlib.pyplot as plt
from torch.optim import Adam
from utils.pipeline import forward, get_latent_mean_std, get_latent_from_text

torch.cuda.empty_cache()

In [2]:
lat_mean, lat_std = get_latent_mean_std()
lat_rep = (torch.randn(lat_mean.shape) * lat_std * 0.85 + lat_mean).detach().requires_grad_(True)

In [ ]:
optimizer = Adam(params=[lat_rep],
                 lr=0.001, 
                 maximize=True)

In [ ]:
camera_params = {
        "camera_distance": 1.42,
        "camera_angle": 55.,
        "focal_length": 3.15,
        "max_ray_length": (0.5 + 1) * 3.15 + 1.5,
        # Image
        "resolution_y": 100,
        "resolution_x": 100
    }

phong_params = {
    "ambient_coeff": 0.67,
    "diffuse_coeff": 0.79,
    "specular_coeff": 0.3,  # 0.63,
    "shininess": 1.,
    # Colors
    "object_color": torch.tensor([0.61, 0.61, 0.61]),  # torch.tensor([0.63, 0.17, 0.78]),
    "background_color": torch.tensor([0., 0., 0.])  # torch.tensor([0.35, 0.94, 0.26])
}

light_params = {
    "amb_light_color": torch.tensor([0.57, 0.07, 0.69]),
    # light 1
    "light_intensity_1": 0.,  # 1.1,
    "light_color_1": torch.tensor([0.88, 0.99, 0.74]),
    "light_dir_1": torch.tensor([-0.41, -0.51, -0.76]),
    # light p
    "light_intensity_p": 1.,
    "light_color_p": torch.tensor([0.88, 0.99, 0.74]),
    "light_pos_p": torch.tensor([2., 0., 2.])
}

In [ ]:
score = forward(lat_rep, "untextured render of a face", camera_params, phong_params, light_params)

In [ ]:

score[0].backward()

In [ ]:
init_latent = torch.load("../woman_latent")

In [6]:
hparams = {
        'resolution': 120,
        'n_iterations': 50,
        'optimizer_lr': 0.002,
        'lr_scheduler_factor': 0.5,
        'lr_scheduler_patience': 2,
        'lr_scheduler_min_lr': 1e-5
    }
#torch.cuda.memory._record_memory_history(True)
best_latent, best_score, hist = get_latent_from_text("A woman", hparams)
#torch.cuda.memory._record_memory_history(False)
#torch.cuda.memory._dump_snapshot("../memory_snapshot.pickle")
print(best_score)

  0%|          | 0/50 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 218.00 MiB (GPU 0; 10.91 GiB total capacity; 1.56 GiB already allocated; 95.44 MiB free; 1.76 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
latent = None

In [ ]:
plt.plot(torch.tensor(hist["scores"]).numpy())

In [ ]:
torch.save(latent, "../woman_latent")

In [ ]:
torch.save(torch.stack(hist["latents"]), "latent_history")

In [ ]:
torch.save(torch.stack(hist["images"]), "render_history")

In [ ]:
plt.imshow(hist["images"][-1].detach().numpy())
plt.axis('off')  # Turn off axes
plt.show()
#plt.savefig(f"optim_img/high_lr_{i}", bbox_inches='tight')

In [ ]:
for i, im in enumerate(hist["images"]):
    plt.imshow(im.detach().numpy())
    plt.axis('off')  # Turn off axes
    #plt.show()
    plt.savefig(f"../optim_img/{i}", bbox_inches='tight')